<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/%E5%B8%A6%E4%BC%9A%E8%AF%9DID%E8%BE%93%E5%85%A5%E5%92%8C%E8%BE%93%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

··········


In [2]:
#!pip install --upgrade --quiet  langchain-google-genai pillow
#!pip install langchain langchain_community
#!pip install --upgrade httpx httpx-sse PyJWT

from langchain_community.chat_models import ChatZhipuAI


from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00

In [4]:
#字典输入，消息输出
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
prompt = ChatPromptTemplate.from_messages(
[
      (
          "system",
          "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
      ),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{input}"),
    ]
)

runnable = prompt | llm
store = {}

def get_session_history(session_id: str) -> RunnableWithMessageHistory:
    #为 session_id 创建一个新的键，并将其值设置为一个 ChatMessageHistory 类的实例
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)


In [6]:
runnable_with_history.invoke(
    {"language": "chinese", "input": "hi im bob!"},
    config={"configurable": {"session_id": "2"}},
)



AIMessage(content='你好，鲍勃！有什么可以帮您的？', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 51, 'total_tokens': 63}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-caadf567-9fcf-4456-8c58-23ce44959c96-0')

In [8]:
runnable_with_history.invoke(
    {"language": "chinese", "input": "whats my name?"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='你的名字是Bob。需要我帮你做什么吗？', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 91, 'total_tokens': 103}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-2d5727b3-a597-4cf6-8c38-5d7f2d7ac9d6-0')

In [9]:
runnable_with_history.invoke(
    {"language": "chinese", "input": "whats my name?"},
    config={"configurable": {"session_id": "3a"}},
)

AIMessage(content='您好！请问您需要什么帮助？', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 28, 'total_tokens': 38}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-6a8190d0-63c7-41f5-a60c-87c5d053e06a-0')